In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import time
import pandas as pd
from pandas import json_normalize
from datetime import datetime
import json


In [2]:
def scrolling_and_cleacking(driver_):
    wait = WebDriverWait(driver_, 10)

    try:
    # Oczekiwanie na pojawienie się przycisku "Akceptuj" i kliknięcie go
        akceptuj_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#onetrust-accept-btn-handler')))
        akceptuj_button.click()
    except Exception as e:
        print("Nie udało się znaleźć lub kliknąć przycisku 'Akceptuj':", e)

    wait = WebDriverWait(driver_, 10)

    while True:
        try:
        # Sprawdzenie, czy przycisk "Pokaż kolejne oferty" nadal jest obecny na stronie
            load_more_button = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'button.tw-btn.tw-btn-primary.tw-px-8.tw-block.tw-btn-xl'))
            )
        # Przewinięcie do przycisku i kliknięcie go
            driver_.execute_script("arguments[0].scrollIntoView();", load_more_button)
            time.sleep(2)
            load_more_button.click()
            print("Kliknięto")
            time.sleep(2)
        except (NoSuchElementException, TimeoutException):
            print("Nie można już znaleźć przycisku 'Pokaż kolejne oferty'. Zakończenie.")
            break
        except ElementClickInterceptedException:
            driver_.execute_script("window.scrollBy(0, 100);")
            time.sleep(2)

    wait = WebDriverWait(driver_, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'nfj-postings-list')))
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.tw-mr-2 [data-cy^="salary "]')))

In [3]:
def offer_salary_info(offer_):
    salary_element = offer_.select_one('[class="tw-mb-0"]')
    salary_text = salary_element.text.strip().replace('\xa0', '')
    if salary_text != "Sprawdź SalaryMatch":
        salary_info = {
        'low': salary_text.split(" ")[0],
        'high': salary_text.split(" ")[3],
        'currency': salary_text.split(" ")[-1]
        }
    else:
            salary_info = "Brak widełki"
    
    return salary_info

In [4]:
def offer_location_info(offer_soup_):
    translate_city = {
    "Warsaw": "Warszawa",
    "Wroclove": "Wrocław",
    "Krakow": "Kraków",
    "Cracow": "Kraków",
    "Gdansk": "Gdańsk",
    "Poznan": "Poznań",
    "Lodz": "Łódź",
    "Bialystok": "Białystok"
    }
    if offer_soup_.select_one("[commondatacyclick]").text.strip() == "Praca zdalna":
        location_info = "Praca zdalna"
    elif len(offer_soup_.select('a span[class="ng-star-inserted"]')) == 1:
        location = offer_soup_.select_one('a span[class="ng-star-inserted"]').text.strip()
        location_info = translate_city.get(location, location)
    else:
        locations = [element.text.strip() for element in offer_soup_.select('a span[class="ng-star-inserted"]')]
        translated_locations = list({translate_city.get(location, location) for location in locations})
        location_info = translated_locations if len(translated_locations) > 1 else translated_locations[0]
    return location_info

In [5]:
def necessery_technology(offer_soup_):
    technology_info = [element.text.strip() for element in offer_soup_.select('[data-cy="JobOffer_Category"], [branch="musts"] span.ng-star-inserted, li[_ngcontent-serverapp-c1062005900].tw-btn.tw-btn-xs.tw-text-sm.tw-font-normal.tw-text-teal.tw-border-2.tw-border-teal.text-truncate.ng-star-inserted')]
    return technology_info

In [6]:
def nice_to_have_technology(offer_soup_):
    if offer_soup_.select_one('#posting-nice-to-have') != None:
        nice_to_have_info = [element.text.strip() for element in offer_soup_.select('#posting-nice-to-have [id*="item-tag"]')]
    else: 
        nice_to_have_info = "Brak"
    return nice_to_have_info

In [7]:
def experience(offer_soup_):
    if offer_soup_.select_one('#posting-seniority span') != None:
        experience_level_info = offer_soup_.select_one('#posting-seniority span').text.strip()
    else: 
        experience_level_info = "Brak"
    return experience_level_info

In [8]:
def offer_exists(offer, records):
    categories = ["data analyst", "data scientist", "data engineer"]
    for category in categories:
        if category in records:
            for existing_offer in records[category]:
                if (existing_offer['name'] == offer['name'] and
                    existing_offer['company'] == offer['company'] and
                    existing_offer['experience level'] == offer['experience level'] and
                    existing_offer['technology'] == offer['technology'] and
                    existing_offer['nice to have'] == offer['nice to have'] and
                    existing_offer['location'] == offer['location'] and
                    existing_offer['salary'] == offer['salary']):
                    return True
    return False

In [9]:
job_titels = ["analyst", "scientist", "engineer"]
records = {}
offers = []


for title in job_titels:
    driver = webdriver.Chrome()
    driver.get("https://link_to_a_job_search_website_data%20{stanowisko}%27".replace("{stanowisko}", title))

    scrolling_and_cleacking(driver)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    for section in soup.select('nfj-postings-list')[:2]:
        for offer in section.select('.posting-list-item'):
            offer_link = offer.get('href')
            offer_page = requests.get(f"https://link_to_a_job_search_website{offer_link}")
            offer_soup = BeautifulSoup(offer_page.text, "html.parser")                    

            new_offer = {
            'name': offer.select_one("h3.posting-title__position").text.strip(),
            'company': offer.select_one("h4.company-name").text.strip(),
            'experience level': experience(offer_soup),
            'technology': necessery_technology(offer_soup),
            'nice to have': nice_to_have_technology(offer_soup),
            'location': offer_location_info(offer_soup),
            'salary': offer_salary_info(offer)
            }

            if not offer_exists(new_offer, records):
                offers.append(new_offer)

    records["data " + title] = offers
    offers = []

driver.quit()

Kliknięto
Kliknięto
Kliknięto
Kliknięto
Nie można już znaleźć przycisku 'Pokaż kolejne oferty'. Zakończenie.
Nie można już znaleźć przycisku 'Pokaż kolejne oferty'. Zakończenie.
Kliknięto
Kliknięto
Nie można już znaleźć przycisku 'Pokaż kolejne oferty'. Zakończenie.


In [11]:
# Zapisanie do json

path = os.path.join("..", "data", "interim")
current_date = datetime.today().strftime('%Y_%m_%d')
with open(os.path.join(path,f'job offers {current_date}.json'), 'w', encoding='utf-8') as file:
    json.dump(records, file, ensure_ascii=False, indent=4)

In [12]:
# Zapisanie do CSV
all_records = []

for category, values in records.items():
    for record in values:
        record['category'] = category
        all_records.append(record)

df = pd.json_normalize(all_records, sep='_')

print(df.head())

path = os.path.join("..", "data", "interim")

os.makedirs(path, exist_ok=True)

current_date = datetime.today().strftime('%Y_%m_%d')
filename = os.path.join(path, f"job_offers_{current_date}.csv")
df.to_csv(filename, sep=';', encoding='utf-8', index=False)

print(f"Saved all job offers to: {filename}")


                                  name             company experience level  \
0                         Data Analyst                 XTB              Mid   
1                     Data Analyst CRM          Connectis_           Junior   
2  Customer Master Data Senior Analyst   Devire Sp. z o.o.           Senior   
3                         Data analyst            Uncapped              Mid   
4                  Senior Data Analyst  Brainly Sp. z o.o.           Senior   

                                          technology  \
0  [Data, Python, SQL, Data base, Polski, Angielski]   
1           [Data, CRM, Python, SQL, Angielski (B2)]   
2  [Data, T-SQL, SAP, Excel, Communication skills...   
3  [Data, SQL, Problem solving, Critical thinking...   
4  [Data, SQL, Google Analytics, Tableau, BigQuer...   

                                        nice to have      location  \
0      [Microsoft Azure, Databricks, Spark, PySpark]  Praca zdalna   
1                                               